## Intrusion Detection using Adaboost and LightGBM

## Nitin Vankadari

In [1]:
import matplotlib
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns
import sklearn
%matplotlib inline

In [2]:
from sklearn import model_selection
from sklearn.model_selection import StratifiedKFold, cross_val_score

In [3]:
dataset=pd.read_csv("Train_Test_Windows_10.csv")

FileNotFoundError: [Errno 2] File b'Train_Test_Windows_10.csv' does not exist: b'Train_Test_Windows_10.csv'

In [ ]:
dataset.head()

In [ ]:
dataset.info()

In [ ]:
dataset.describe()

In [ ]:
dataset.shape

In [ ]:
dataset['type'].value_counts()

In [ ]:
plt.figure(figsize=(30,15))
sns.heatmap(dataset.corr())

In [ ]:
dataset=dataset.apply(lambda x: x.apply (lambda x: np.nan if x==' ' else x))

In [ ]:
n=dataset.isnull().sum()
n[n!=0]

In [ ]:
dataset=dataset.dropna()

In [ ]:
useless_col=[]
for i in dataset.columns:
    if(len(dataset[i].unique())==1):
        useless_col.append(i)

print("Total useless columns= ",len(useless_col))
dataset.drop(useless_col,1,inplace=True)

In [ ]:
boolean_col=[]
for i in dataset.columns:
    if(len(dataset[i].unique())==2):
        boolean_col.append(i)

print("Total boolean columns= ",len(boolean_col))


In [ ]:
for i in dataset.columns:
    if(len(dataset[i].unique())==2):
        print(dataset[i].value_counts())


In [ ]:
dataset['Memory System Code Total Bytes']=dataset['Memory System Code Total Bytes'].apply(lambda x: 1 if x==7155712 else 0)

In [ ]:
dataset.info()

In [ ]:
float_col= dataset.select_dtypes(exclude='int64').drop('type',1).columns
dataset[float_col]=dataset[float_col].astype('float')

In [ ]:
dataset.info()

In [ ]:
from sklearn.preprocessing import LabelEncoder
LE = LabelEncoder()
dataset["type_Enc"]=LE.fit_transform(dataset.type)
dataset.drop(['type'],axis=1,inplace=True)

In [ ]:
dataset['type_Enc'].value_counts()

In [ ]:
y=dataset['type_Enc']
x=dataset.drop(['type_Enc'], axis=1)

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
dataset_name=dataset.iloc[:,:-1]
cols = dataset_name.columns
sc_dataset = scaler.fit_transform(x)
# turn the result back to a dataframe
dataset = pd.DataFrame(sc_dataset, columns = cols)

In [ ]:
dataset

In [ ]:
import imblearn
from imblearn.over_sampling import RandomOverSampler
ro=RandomOverSampler(sampling_strategy='auto',random_state=1)
Xro,yro=ro.fit_resample(x,y)

In [ ]:
yro.value_counts()

In [ ]:
X_train, X_test, y_train, y_test = model_selection.train_test_split(Xro,yro, test_size=0.4, random_state=42, stratify=yro)

In [28]:
from sklearn.ensemble import AdaBoostClassifier
model = AdaBoostClassifier(random_state=42)
model.fit(X_train,y_train)
model.score(X_test,y_test)

0.375

In [29]:
from sklearn.model_selection import StratifiedKFold, cross_val_score
from sklearn.ensemble import AdaBoostClassifier
from numpy import mean
model = AdaBoostClassifier(random_state=42)
def evaluate_model(model):
    cv = StratifiedKFold(n_splits=10,shuffle=True, random_state=42)
    scores = cross_val_score(model, X_train, y_train, scoring='accuracy', cv=cv, n_jobs=-1, verbose = 1, error_score='raise')
    return scores
mean(evaluate_model(model))

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   6 out of  10 | elapsed:  1.4min remaining:   57.4s
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:  1.8min finished


0.4248770053475936

In [31]:
model.fit(X_train,y_train)
y_pred = model.predict(X_test)
def get_classification_report(y_test, y_pred):
    from sklearn import metrics
    report = metrics.classification_report(y_test, y_pred, output_dict=True)
    df_classification_report = pd.DataFrame(report).transpose()
    df_classification_report = df_classification_report.sort_values(by=['f1-score'], ascending=False)
    return df_classification_report
get_classification_report(y_test, y_pred)

C:\Users\nitva\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


,precision,recall,f1-score,support
1,1.000000,1.000,1.000000,3896.000
4,1.000000,1.000,1.000000,3896.000
accuracy,0.375000,0.375,0.375000,0.375
0,0.166667,1.000,0.285714,3896.000
macro avg,0.270833,0.375,0.285714,31168.000
weighted avg,0.270833,0.375,0.285714,31168.000
2,0.000000,0.000,0.000000,3896.000
3,0.000000,0.000,0.000000,3896.000
5,0.000000,0.000,0.000000,3896.000
6,0.000000,0.000,0.000000,3896.000


In [32]:
from sklearn.metrics import confusion_matrix
confmetric = confusion_matrix(y_test,y_pred)
confmetric

array([[3896,    0,    0,    0,    0,    0,    0,    0],
       [   0, 3896,    0,    0,    0,    0,    0,    0],
       [3896,    0,    0,    0,    0,    0,    0,    0],
       [3896,    0,    0,    0,    0,    0,    0,    0],
       [   0,    0,    0,    0, 3896,    0,    0,    0],
       [3896,    0,    0,    0,    0,    0,    0,    0],
       [3896,    0,    0,    0,    0,    0,    0,    0],
       [3896,    0,    0,    0,    0,    0,    0,    0]], dtype=int64)

In [4]:
import time 
time_start=time.clock()
import lightgbm as lgbm
from lightgbm import LGBMClassifier
model = LGBMClassifier(random_state=42)
model.fit(X_train,y_train)
model.score(X_test,y_test)
time_elapsed=(time.clock()-time_start)
time_elapsed

C:\Users\hriti\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead
  


NameError: name 'X_train' is not defined

In [29]:
from lightgbm import LGBMClassifier
model = LGBMClassifier(random_state=42)
def evaluate_model(model):
    cv = StratifiedKFold(n_splits=10,shuffle=True, random_state=42)
    scores = cross_val_score(model, X_train, y_train, scoring='accuracy', cv=cv, n_jobs=-1, verbose = 1, error_score='raise')
    return scores
mean(evaluate_model(model))

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:  4.1min finished


1.0

In [30]:
model.fit(X_train,y_train)
y_pred = model.predict(X_test)
def get_classification_report(y_test, y_pred):
    from sklearn import metrics
    report = metrics.classification_report(y_test, y_pred, output_dict=True)
    df_classification_report = pd.DataFrame(report).transpose()
    df_classification_report = df_classification_report.sort_values(by=['f1-score'], ascending=False)
    return df_classification_report
get_classification_report(y_test, y_pred)

,precision,recall,f1-score,support
0,1.0,1.0,1.0,3896.0
1,1.0,1.0,1.0,3896.0
2,1.0,1.0,1.0,3896.0
3,1.0,1.0,1.0,3896.0
4,1.0,1.0,1.0,3896.0
5,1.0,1.0,1.0,3896.0
6,1.0,1.0,1.0,3896.0
7,1.0,1.0,1.0,3896.0
accuracy,1.0,1.0,1.0,1.0
macro avg,1.0,1.0,1.0,31168.0


In [31]:
from sklearn.metrics import confusion_matrix
confmetric = confusion_matrix(y_test,y_pred)
confmetric

array([[3896,    0,    0,    0,    0,    0,    0,    0],
       [   0, 3896,    0,    0,    0,    0,    0,    0],
       [   0,    0, 3896,    0,    0,    0,    0,    0],
       [   0,    0,    0, 3896,    0,    0,    0,    0],
       [   0,    0,    0,    0, 3896,    0,    0,    0],
       [   0,    0,    0,    0,    0, 3896,    0,    0],
       [   0,    0,    0,    0,    0,    0, 3896,    0],
       [   0,    0,    0,    0,    0,    0,    0, 3896]], dtype=int64)

In [31]:
params = {'AB':{'learning_rate': [0.0001, 0.1, 1.0], 'n_estimators':(10,100,500)},'LGBM':{'max_depth':(3,8), 'n_estimators': range(50,201,100),'learning_rate': [0.05, 0.1, 0.2],'subsample':[0.5,0.9],'num_leaves': (15,35)}}

In [32]:
from sklearn.model_selection import GridSearchCV
best_params = params
names= list()
models = {'AB':AdaBoostClassifier(random_state=42),'LGBM':LGBMClassifier(random_state=42)}
for name, param_grid, model in zip(params.keys(), params.values(), models.values()):
    gscv = GridSearchCV(model, param_grid, n_jobs=-1, verbose=3, cv=4)
    gscv.fit(X_train,y_train)
    names.append(name)
    best_params[name] = gscv.best_params_
    print(name)
    print("best score:",gscv.best_score_)
    print("best params:",gscv.best_params_)

Fitting 4 folds for each of 9 candidates, totalling 36 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  24 tasks      | elapsed: 18.5min
[Parallel(n_jobs=-1)]: Done  36 out of  36 | elapsed: 27.4min finished


AB
best score: 0.9780544147843944
best params: {'learning_rate': 0.1, 'n_estimators': 500}
Fitting 4 folds for each of 48 candidates, totalling 192 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  24 tasks      | elapsed:  3.4min
[Parallel(n_jobs=-1)]: Done 120 tasks      | elapsed: 24.6min
[Parallel(n_jobs=-1)]: Done 192 out of 192 | elapsed: 39.5min finished


LGBM
best score: 1.0
best params: {'learning_rate': 0.05, 'max_depth': 3, 'n_estimators': 50, 'num_leaves': 15, 'subsample': 0.5}


In [30]:
from sklearn import model_selection
from sklearn.model_selection import StratifiedKFold,KFold,RepeatedStratifiedKFold,cross_val_score
from xgboost import XGBClassifier
from numpy import mean

In [31]:
model = XGBClassifier(random_state=42)
model.fit(X_train, y_train)
model.score(X_test,y_test)

1.0